In [2]:
# !pip install PyMuPDF
from PyPDF2 import PdfReader
import os
import re
import glob
import pandas as pd
import fitz  # PyMuPDF

In [ ]:
def extraer_tabla_contenidos(pdf_path):
    # Abre el archivo PDF
    pdf_documento = fitz.open(pdf_path)

    # Busca el índice de la página que contiene la tabla de contenidos
    indice_pagina_contenidos = -1
    for num_pagina in range(pdf_documento.page_count):
        pagina = pdf_documento[num_pagina]
        texto_pagina = pagina.get_text()
        if "Contents" in texto_pagina:
            indice_pagina_contenidos = num_pagina
            break

    # Si no se encuentra la tabla de contenidos, maneja el caso aquí según tus necesidades

    # Extrae el texto de la página de la tabla de contenidos
    texto_contenidos = ""
    i = indice_pagina_contenidos
    while i < min(indice_pagina_contenidos + 2, pdf_documento.page_count):
        texto_contenidos += pdf_documento[i].get_text()
        i += 1
    # Cierra el documento
    pdf_documento.close()

    return texto_contenidos


In [ ]:
def encontrar_pagina_seccion(texto_contenidos, patron_seccion):
    """
    Encuentra la página de inicio de una sección específica en el texto de la tabla de contenidos.
    """
    lineas = texto_contenidos.split('\n')
    for i, linea in enumerate(lineas):
        if patron_seccion.search(linea):
            # Asume que el número de página está en las líneas siguientes
            for j in range(i+1, min(i+6, len(lineas))):  # Busca en las siguientes 5 líneas como máximo
                match_numero = re.search(r'\d+', lineas[j])
                if match_numero:
                    return int(match_numero.group(0))
    return None


In [77]:
def encontrar_pagina_real(pdf_path, numero_pagina_texto):
    """
    Encuentra la página real del PDF basada en el número de página que aparece en el texto.
    """
    pdf_documento = fitz.open(pdf_path)
    texto_completo = ''
    paginas_texto_a_pdf = {}
    
    # Extraer el texto de cada página y buscar el número de página del texto
    for num_pagina in range(pdf_documento.page_count):
        pagina = pdf_documento.load_page(num_pagina)
        texto_pagina = pagina.get_text()
        # Guardar el texto completo para una posible búsqueda global (opcional)
        texto_completo += f"\n{texto_pagina}"
        # Buscar el patrón específico en el texto de la página
        if f"{numero_pagina_texto}\nBIS" in texto_pagina:
            paginas_texto_a_pdf[numero_pagina_texto] = num_pagina + 1  # Las páginas en PDF generalmente comienzan en 1
            break  # Asumiendo que cada número de página del texto es único
    
    pdf_documento.close()
    
    # Devolver el número de página en el PDF (0 si no se encuentra)
    return paginas_texto_a_pdf.get(numero_pagina_texto, 0)

In [95]:
# Supongamos que tienes una lista de rutas de archivos PDF
rutas_pdf = [rf"C:\Users\Diego\Documents\PUCP\WebScrapping\OCR\ar20{i}e.pdf" for i in range(24)]

# Lista para almacenar el texto extraído
textos_extraidos = []
T = []
# Extraer texto de cada PDF

for pdf_path in rutas_pdf:
    with open(pdf_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        n_pages  = len( pdf_reader.pages )
        pdf_full = [ pdf_reader.pages[ i ].extract_text() for i in range( n_pages ) ]
        
        # Extrae la tabla de contenidos
        tabla_contenidos = extraer_tabla_contenidos(pdf_path)
    
        # # Imprime la tabla de contenidos
        # print(tabla_contenidos)
    
        # Prepara el patrón de búsqueda para la sección de interés
        patron_seccion = re.compile(r"\s*(Monetary policy)", re.IGNORECASE)
    
        # Encuentra la página de inicio de la sección de política monetaria
        pagina_inicio_seccion = encontrar_pagina_seccion(tabla_contenidos, patron_seccion)
    
        # if pagina_inicio_seccion:
        #     print(f"Sección de política monetaria encontrada en la página {pagina_inicio_seccion}.")
        # else:
        #     print("Sección de política monetaria no encontrada en la tabla de contenidos.")
    
        
        numero_pagina_texto = pagina_inicio_seccion  # El número de página que encontraste en la tabla de contenidos
    
        # Encontrar la página real en el PDF
        pagina_pdf_real = encontrar_pagina_real(pdf_path, numero_pagina_texto)

        # if pagina_pdf_real:
        #     print(f"La sección comienza en la página real del PDF: {pagina_pdf_real}.")
        # else:
        #     print("No se encontró la página real en el PDF para el número de página dado.")

        # extracted_text = pdf_text[ pagina_pdf_real+1 ]
    
        # textos_extraidos.append(' '.join(page.extract_text() for page in pdf_reader.pages))
    
        textos_extraidos.append(pdf_full[ pagina_pdf_real+1 ])
for i in range(15, 24):
    T.append(2000 + i)
    
# Crear un DataFrame con una columna llamada 'Texto'
df = pd.DataFrame({'Año': T,'Política Monetaria': textos_extraidos})
        
df 